# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/olivereve/Documents/GitHub/python-api-challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sinop,41.7711,34.8709,10.13,90,40,1.08,TR,1684449002
1,1,cabitan,12.2943,123.3412,28.32,74,96,2.41,PH,1684449003
2,2,jamestown,42.0970,-79.2353,20.85,28,0,2.57,US,1684448784
3,3,castro,-24.7911,-50.0119,11.73,74,0,0.87,BR,1684449005
4,4,sydney mines,46.2367,-60.2177,8.33,47,20,8.23,CA,1684449005


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display
import geoviews as gv
import holoviews as hv
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    title= 'OSM',
    size= 'Humidity',
    color= 'City',
    frame_width = 700,
    frame_height = 700,
    alpha = 1

)

# Display the map
display(map_plot)

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_weather = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_dropped = city_weather.dropna()

# Display sample data
city_dropped

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,resistencia,-27.4606,-58.9839,23.03,76,0,1.54,AR,1684449008
87,87,concordia,-31.3930,-58.0209,21.55,78,0,1.34,AR,1684449059
140,140,saint-pierre,-21.3393,55.4781,23.82,83,0,2.57,RE,1684449093
212,212,loralai,30.3705,68.5980,21.89,28,0,3.57,PK,1684449142
290,290,awjilah,29.1081,21.2869,24.33,19,0,3.28,LY,1684449187
312,312,gunjur,13.2019,-16.7339,25.04,83,0,4.12,GM,1684449199
314,314,saint anthony,45.0205,-93.2180,22.12,70,0,3.09,US,1684449200
368,368,holt,42.6406,-84.5153,21.06,31,0,3.09,US,1684449101
407,407,morondava,-20.2833,44.2833,21.51,62,0,2.59,MG,1684449258
455,455,angoche,-16.2325,39.9086,23.29,82,0,2.32,MZ,1684449284


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_dropped.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,resistencia,AR,-27.4606,-58.9839,76,
87,concordia,AR,-31.3930,-58.0209,78,
140,saint-pierre,RE,-21.3393,55.4781,83,
212,loralai,PK,30.3705,68.5980,28,
290,awjilah,LY,29.1081,21.2869,19,
312,gunjur,GM,13.2019,-16.7339,83,
314,saint anthony,US,45.0205,-93.2180,70,
368,holt,US,42.6406,-84.5153,31,
407,morondava,MG,-20.2833,44.2833,62,
455,angoche,MZ,-16.2325,39.9086,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'radius': radius,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
resistencia - nearest hotel: Residencial Bariloche
concordia - nearest hotel: D'Charruas
saint-pierre - nearest hotel: Tropic Hotel
loralai - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
gunjur - nearest hotel: Footsteps Eco Lodge
saint anthony - nearest hotel: Delta Hotels by Marriott
holt - nearest hotel: Causeway Bay Lansing
morondava - nearest hotel: Hotel Sharon Menabe
angoche - nearest hotel: Hotel Quirimbas
umm lajj - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
8,resistencia,AR,-27.4606,-58.9839,76,Residencial Bariloche
87,concordia,AR,-31.3930,-58.0209,78,D'Charruas
140,saint-pierre,RE,-21.3393,55.4781,83,Tropic Hotel
212,loralai,PK,30.3705,68.5980,28,No hotel found
290,awjilah,LY,29.1081,21.2869,19,No hotel found
312,gunjur,GM,13.2019,-16.7339,83,Footsteps Eco Lodge
314,saint anthony,US,45.0205,-93.2180,70,Delta Hotels by Marriott
368,holt,US,42.6406,-84.5153,31,Causeway Bay Lansing
407,morondava,MG,-20.2833,44.2833,62,Hotel Sharon Menabe
455,angoche,MZ,-16.2325,39.9086,82,Hotel Quirimbas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    title= 'OSM',
    size= 'Humidity',
    color= 'City',
    frame_width = 700,
    frame_height = 500,
    alpha = 1,
    hover_cols = ['City', 'Country', 'Hotel Name']

)

# Display the map
display(map_plot)



:Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)